# Q2: Data Cleaning

**Phase 3:** Data Cleaning & Preprocessing  
**Points: 9 points**

**Focus:** Handle missing data, outliers, validate data types, remove duplicates.

**Lecture Reference:** Lecture 11, Notebook 1 ([`11/demo/01_setup_exploration_cleaning.ipynb`](https://github.com/christopherseaman/datasci_217/blob/main/11/demo/01_setup_exploration_cleaning.ipynb)), Phase 3. Also see Lecture 05 (data cleaning).

---

## Setup

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load data from Q1 (or directly from source)
df = pd.read_csv('data/beach_sensors.csv')
# If you saved cleaned data from Q1, you can load it:
# df = pd.read_csv('output/q1_exploration.csv')  # This won't work - load original

---

## Objective

Clean the dataset by handling missing data, outliers, validating data types, and removing duplicates.

**Time Series Note:** For time series data, forward-fill (`ffill()`) is often appropriate for missing values since sensor readings are continuous. However, you may choose other strategies based on your analysis.

---

## Required Artifacts

You must create exactly these 3 files in the `output/` directory:

### 1. `output/q2_cleaned_data.csv`
**Format:** CSV file
**Content:** Cleaned dataset with same structure as original (same columns)
**Requirements:**
- Same columns as original dataset
- Missing values handled (filled, dropped, or imputed)
- Outliers handled (removed, capped, or transformed)
- Data types validated and converted
- Duplicates removed
- **Sanity check:** Dataset should retain most rows after cleaning (at least 1,000 rows). If you're removing more than 50% of data, reconsider your strategy—imputation is usually preferable to dropping rows for this dataset.
- **No index column** (save with `index=False`)

### 2. `output/q2_cleaning_report.txt`
**Format:** Plain text file
**Content:** Detailed report of cleaning operations
**Required information:**
- Rows before cleaning: [number]
- Missing data handling method: [description]
  - Which columns had missing data
  - Method used (drop, forward-fill, impute, etc.)
  - Number of values handled
- Outlier handling: [description]
  - Detection method (IQR, z-scores, domain knowledge)
  - Which columns had outliers
  - Method used (remove, cap, transform)
  - Number of outliers handled
- Duplicates removed: [number]
- Data type conversions: [list any conversions]
- Rows after cleaning: [number]

**Example format:**
```
DATA CLEANING REPORT
====================

Rows before cleaning: 50000

Missing Data Handling:
- Water Temperature: 2500 missing values (5.0%)
  Method: Forward-fill (time series appropriate)
  Result: All missing values filled
  
- Air Temperature: 1500 missing values (3.0%)
  Method: Forward-fill, then median imputation for remaining
  Result: All missing values filled

Outlier Handling:
- Water Temperature: Detected 500 outliers using IQR method (3×IQR)
  Method: Capped at bounds [Q1 - 3×IQR, Q3 + 3×IQR]
  Bounds: [-5.2, 35.8]
  Result: 500 values capped

Duplicates Removed: 0

Data Type Conversions:
- Measurement Timestamp: Converted to datetime64[ns]

Rows after cleaning: 50000
```

### 3. `output/q2_rows_cleaned.txt`
**Format:** Plain text file
**Content:** Single integer number (total rows after cleaning)
**Requirements:**
- Only the number, no text, no labels
- No whitespace before or after
- Example: `50000`

---

## Requirements Checklist

- [ ] Missing data handling strategy chosen and implemented
- [ ] Outliers detected and handled (IQR method, z-scores, or domain knowledge)
- [ ] Data types validated and converted
- [ ] Duplicates identified and removed
- [ ] Cleaning decisions documented in report
- [ ] All 3 required artifacts saved with exact filenames

---

## Your Approach

1. **Handle missing data** - Choose appropriate strategy (drop, forward-fill, impute) based on data characteristics
2. **Detect and handle outliers** - Use IQR method or z-scores; decide whether to remove, cap, or transform
3. **Validate data types** - Ensure numeric and datetime columns are properly typed
4. **Remove duplicates**
5. **Document and save** - Write detailed cleaning report explaining your decisions

---

## Decision Points

- **Missing data:** Should you drop rows, impute values, or forward-fill? Consider: How much data is missing? Is it random or systematic? For time series, forward-fill is often appropriate.
- **Outliers:** Are they errors or valid extreme values? Use IQR method or z-scores to detect, then decide: remove, cap, or transform. Document your reasoning.
- **Data types:** Are numeric columns actually numeric? Are datetime columns properly formatted? Convert as needed.

---

## Checkpoint

After Q2, you should have:
- [ ] Missing data handled
- [ ] Outliers addressed
- [ ] Data types validated
- [ ] Duplicates removed
- [ ] All 3 artifacts saved: `q2_cleaned_data.csv`, `q2_cleaning_report.txt`, `q2_rows_cleaned.txt`

---

**Next:** Continue to `q3_data_wrangling.md` for Data Wrangling.


In [2]:
# Check if missing values occur in the same rows
df_clean = df.copy()

cols_with_same_missing = ['Air Temperature', 'Wet Bulb Temperature', 'Rain Intensity', 'Total Rain', 'Precipitation Type', 'Barometric Pressure', 'Heading']
missing_mask = df_clean[cols_with_same_missing].isnull()
print(f"Rows where ALL are missing: {missing_mask.all(axis=1).sum()}")
print(f"Rows where ANY are missing: {missing_mask.any(axis=1).sum()}")

NameError: name 'df' is not defined

In [ ]:
#impute missing values using forward fill
df_copy = df.copy()
df_copy["Air Temperature"] = df_copy["Air Temperature"].fillna(method="ffill")
df_copy["Wet Bulb Temperature"] = df_copy["Wet Bulb Temperature"].fillna(method="ffill")
df_copy["Rain Intensity"] = df_copy["Rain Intensity"].fillna(method="ffill")
df_copy["Total Rain"] = df_copy["Total Rain"].fillna(method="ffill")
df_copy["Precipitation Type"] = df_copy["Precipitation Type"].fillna(method="ffill")
df_copy["Barometric Pressure"] = df_copy["Barometric Pressure"].fillna(method="ffill")
df_copy["Heading"] = df_copy["Heading"].fillna(method="ffill")


#handle outliers
num_cols = ["Air Temperature", "Wet Bulb Temperature","Humidity", 
            "Rain Intensity","Interval Rain", "Total Rain", "Precipitation Type",
            "Wind Direction", "Wind Speed", "Maximum Wind Speed", "Barometric Pressure",
            "Solar Radiation", "Heading", "Battery Life"]
outlier_counts = {}
for var in num_cols:
    q1 = df_copy[var].quantile(0.25)
    q3 = df_copy[var].quantile(0.75)
    mid = q3 - q1
    lb = q1 - 1.5 * mid
    ub = q3 + 1.5 * mid
    num_outliers = ((df_copy[var] < lb) | (df_copy[var] > ub)).sum()
    outlier_counts[var] = num_outliers 
    df_copy[var] = np.where(
        df_copy[var] > ub, ub,
        np.where(df_copy[var] < lb, lb, df_copy[var])
    )

#check types
df_copy["Measurement Timestamp"] = pd.to_datetime(df_copy["Measurement Timestamp"])

map = {0: "No Precipitation", 60: "Liquid Precipitation", 70: "Solid Precipitation", 40: "Unspecified Precipitation"}
df_copy["Precipitation Type"] = df_copy["Precipitation Type"].map(map).astype("category")



#remove duplicates
df_copy = df_copy.drop_duplicates()

df_copy.to_csv("output/q2_cleaned_data.csv", index=False)

/var/folders/tl/bj19ztj54sl9lbx8m4r74x4r0000gq/T/ipykernel_9273/3451036732.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_copy["Air Temperature"] = df_copy["Air Temperature"].fillna(method="ffill")
/var/folders/tl/bj19ztj54sl9lbx8m4r74x4r0000gq/T/ipykernel_9273/3451036732.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_copy["Wet Bulb Temperature"] = df_copy["Wet Bulb Temperature"].fillna(method="ffill")
/var/folders/tl/bj19ztj54sl9lbx8m4r74x4r0000gq/T/ipykernel_9273/3451036732.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_copy["Rain Intensity"] = df_copy["Rain Intensity"].fillna(method="ffill")
/var/folders/tl/bj19ztj54sl9lbx8m4r74x4r0000gq/T/ipykernel_9273/3451036732.py:6: FutureWarning: Series.fill

In [ ]:
##FILE 2

t1 = f"Rows before cleaning: {df.shape[0]}"
t2 = "Missing data handling method: used forward fill to impute all missing values."
t3 = "Air Temperature, Wet Bulb Temperature, Rain Intensity, Total Rain, Precipitation Type, Barometric Pressure, and Heading had missing data"
t4 = "All but Air Temperature and Barometric Pressure had 75935 values that needed to be filled. Air Temperature had 75, and Barometric Pressure had 146."
t5 = "The outlier detection method used was IQR, and any values that were considered outliers were capped."
t6 = f"Columns and number of outliers before cleaning: {outlier_counts}"
t7 = "0 duplicates found and removed"
t8 = "Measurement Timestamp was made into a datetime variable, and Precipitation Type was made into a categorical variable."
t9 = f"Rows after cleaning: {df_copy.shape[0]}"

texts = [t1,t2,t3,t4,t5,t6,t7,t8,t9]
texts = "\n\n".join(texts)
with open("output/q2_cleaning_report.txt", "w") as f:
    f.write(texts)

In [ ]:
##FILE 3

ret = df_copy.shape[0]
with open("output/q2_rows_cleaned.txt", "w") as f:
    f.write(str(ret))